## Other examples (run for the debug purposes)

In [1]:
import os
import sys

# Fisrt, we add the location of the library to test to the PYTHON path
if sys.platform=="cygwin":
    from cyglibra_core import *
elif sys.platform=="linux" or sys.platform=="linux2":
    from liblibra_core import *
import util.libutil as comn
from libra_py import ERGO_methods
from libra_py import units
from libra_py.workflows.nbra import step2_ergoscf


/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::complex<double>, std::allocator<std::complex<double> > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, s

Define two types of the ErgoSCF execution:

* run1 - do the SCF calculations, for a regular geometry SCF
* run2 - do compute the overlaps only, for the dimer

In [2]:
def run1(EXE, COORDS):
    inp = """#!bin/sh
%s << EOINPUT > /dev/null
spin_polarization = 0
molecule_inline 
%sEOF
basis = "STO-3G"
scf.create_mtx_file_S = 1
scf.create_mtx_files_F = 1
XC.sparse_mode = 1
run "LDA"
EOINPUT
""" % (EXE, COORDS)
    
    #print inp
    return inp

In [3]:
def run2(EXE, COORDS):
    inp = """#!bin/sh
%s << EOINPUT > /dev/null
spin_polarization = 0
molecule_inline 
%sEOF
basis = "STO-3G"
scf.create_basis_func_coord_file = 1
scf.create_mtx_file_S = 1
scf.create_mtx_files_S_and_quit = 1
XC.sparse_mode = 1
run "LDA"
EOINPUT
""" % (EXE, COORDS)
    
    #print inp
    return inp

### 1. Extract geometry and submit an SCF calculation

Here is another example - just to submit the ErgoSCF calculation for a given geometry and run type

In [4]:
EXE = "/mnt/c/cygwin/home/Alexey-user/Soft/ergo-3.8/source/ergo"
R = ERGO_methods.xyz_traj2gen_sp("md1.xyz", 0)
command = run1(EXE, R)
print( command)
os.system("%s" % (command))

#!bin/sh
/mnt/c/cygwin/home/Alexey-user/Soft/ergo-3.8/source/ergo << EOINPUT > /dev/null
spin_polarization = 0
molecule_inline 
 Li      0.00000000       0.00000000       0.00000000
 H      0.00000000       0.00000000       1.33200000
EOF
basis = "STO-3G"
scf.create_mtx_file_S = 1
scf.create_mtx_files_F = 1
XC.sparse_mode = 1
run "LDA"
EOINPUT



0

Using the "find_last_file" to determine which file is the last - presumably it would contain the converged Fock matrix

In [5]:
i, filename = ERGO_methods.find_last_file("F_matrix_", ".mtx")
print( i, filename)

11 F_matrix_11.mtx


We can read the produced matrices - in this case the Fock matrices. And we can specify which section of the matrix we want to read

In [6]:
F = ERGO_methods.get_mtx_matrices(filename)
F.show_matrix("F.txt")

F3 = ERGO_methods.get_mtx_matrices(filename, [0,1,2], [0,1,2])
F3.show_matrix("Fa.txt")

### 2. Run an SCF calculations for a single geometry

Prety-much all of the above in a single step + cleanup

In [7]:
params = {"EXE":"/mnt/c/cygwin/home/Alexey-user/Soft/ergo-3.8/source/ergo", "md_file":"md1.xyz"}
i = 0

E, MO = step2_ergoscf.do_step(i, params, run1)

And we can print out the resulting matrices into a file

In [8]:
E.show_matrix("E0.txt")
MO.show_matrix("MO.txt")

### 3. Calculations of the AO overlaps between different geometries

Get a geometry for a "super-molecule" composed of two snapshots

In [9]:
i = 0
R = ERGO_methods.xyz_traj2gen_ovlp("md1.xyz", i, i+1)
print( R)

 Li      0.00000000       0.00000000       0.00000000
 H      0.00000000       0.00000000       1.33200000
 Li      0.00000000       0.00000000       0.00000000
 H      0.00000000       0.00000000       1.33200000



Here is an example to run the calculation of the overlap matrix between AOs in the two adjacent geometries

In [10]:
params = {"EXE":"/mnt/c/cygwin/home/Alexey-user/Soft/ergo-3.8/source/ergo", "md_file":"md1.xyz"}
i = 0

S11, S22, S12 = step2_ergoscf.do_ovlp(i, params, run2)
S12.show_matrix("S_sub.txt")

In [11]:
#step2_ergoscf.clean()